In [58]:
import pandas as pd
import numpy as np


In [4]:
!pip install transformers


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for PyTorch
Failed to build PyTorch
  Running setup.py install for PyTorch: started
  Running setup.py install for PyTorch: finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [6 lines of output]
  Traceback (most recent call last):
    File "<string>", line 2, in <module>
    File "<pip-setuptools-caller>", line 34, in <module>
    File "C:\Users\yotam\AppData\Local\Temp\pip-install-pnaz4dcs\pytorch_f147a3780d254a7ab8995fddfd61c085\setup.py", line 15, in <module>
      raise Exception(message)
  Exception: You tried to install "pytorch". The package named for PyTorch is "torch"
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for PyTorch
  error: subprocess-exited-with-error
  
  Running setup.py install for PyTorch did not run successfully.
  exit code: 1
  
  [6 lines of output]
  Traceback (most recent call last):
    File "<string>", line 2, in <module>
    File "<pip-setuptools-caller>", line 34, in <module>
    File "C:\Users\yotam\AppData\Local\Te

In [6]:
!pip install torch

Defaulting to user installation because normal site-packages is not writeable
     ------------------------------------ 172.4/172.4 MB 688.9 kB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


# the model

In [206]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer,TFAutoModelForSequenceClassification
import torch

def santiment_prediction(data,col_name,model=None,tokenizer = None):
    if not model:
        model = TFAutoModelForSequenceClassification.from_pretrained("KernAI/stock-news-distilbert",from_pt=True)
    if not tokenizer:
        tokenizer = AutoTokenizer.from_pretrained("KernAI/stock-news-distilbert")
    classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)
    data["santiment_pred"] =data[col_name].apply(lambda sent: list(classifier(sent)[0].values()))
    data["power_pred"] = data['santiment_pred'].apply(lambda a: a[1])
    data['santiment_pred'] = data['santiment_pred'].apply(lambda a: a[0])
    return data
    

In [207]:
data_small = data.sample(25)
data_small = santiment_prediction(data_small,'Sentence')

All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


# another way:
https://www.kaggle.com/code/pratul007/lstm-75-accuracy-word-cloud

# FinancialBERT-Sentiment-Analysis 

In [215]:
from transformers import BertTokenizer, BertForSequenceClassification,TFBertForSequenceClassification
from transformers import pipeline

model = TFBertForSequenceClassification.from_pretrained("ahmedrachid/FinancialBERT-Sentiment-Analysis",num_labels=3,from_pt=True)
tokenizer = BertTokenizer.from_pretrained("ahmedrachid/FinancialBERT-Sentiment-Analysis")

nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
data_small = data.sample(500)
data_small["santiment_pred"] =data_small['Sentence'].apply(lambda sent: list(classifier(sent)[0].values()))
data_small["power_pred"] = data_small['santiment_pred'].apply(lambda a: a[1])
data_small['santiment_pred'] = data_small['santiment_pred'].apply(lambda a: a[0])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [216]:
data_small["right"] = np.where(data_small['Sentiment']==data_small['santiment_pred'],1,0)
data_small["wrong"] = np.where(data_small['Sentiment']==data_small['santiment_pred'],0,1)

data_small["strongly_wrong"] =np.where(data_small['Sentiment']!=data_small['santiment_pred'] ,1,0)
data_small["strongly_wrong"] =np.where(data_small['Sentiment']=='neutral' ,0,data_small["strongly_wrong"])
data_small["strongly_wrong"] =np.where(data_small['santiment_pred']=='neutral' ,0,data_small["strongly_wrong"])

print("right rate: ",data_small["right"].sum()/data_small["right"].count())
print("very wrong rate: ",data_small['strongly_wrong'].sum()/data_small["strongly_wrong"].count())

right rate:  0.596
very wrong rate:  0.028


In [217]:
data_small

,Sentence,Sentiment,santiment_pred,power_pred,right,wrong,strongly_wrong
2561,`` The Government has had a very professional ...,positive,neutral,0.625354,0,1,0
1754,1 Block lamp Harri Koskinen 's light was intro...,positive,positive,0.993958,1,0,0
4599,Operating profit fell to EUR 6.2 mn from EUR 8...,neutral,negative,0.997797,0,1,0
1759,4 January 2011 - Finnish media company Alma Me...,neutral,positive,0.944086,0,1,0
4171,"In Lithuania , operating profit rose to EUR 19...",positive,positive,0.996990,1,0,0
...,...,...,...,...,...,...,...
3683,MegaFon 's subscriber base increased 16.1 % in...,positive,positive,0.997880,1,0,0
2273,The Stena Poseidon is a so-called Panamax tank...,neutral,positive,0.933656,0,1,0
1132,Public services will also be available .,neutral,positive,0.830922,0,1,0
5506,Cargo volume increased by approximately 5 % .,positive,positive,0.995506,1,0,0


# bert-base-multilingual-uncased-sentiment

In [219]:
nlp = pipeline("sentiment-analysis", model='nlptown/bert-base-multilingual-uncased-sentiment')
data_small = data.sample(300)
data_small["santiment_pred"] =data_small['Sentence'].apply(lambda sent: list(classifier(sent)[0].values()))
data_small["power_pred"] = data_small['santiment_pred'].apply(lambda a: a[1])
data_small['santiment_pred'] = data_small['santiment_pred'].apply(lambda a: a[0])
data_small["right"] = np.where(data_small['Sentiment']==data_small['santiment_pred'],1,0)
data_small["wrong"] = np.where(data_small['Sentiment']==data_small['santiment_pred'],0,1)

data_small["strongly_wrong"] =np.where(data_small['Sentiment']!=data_small['santiment_pred'] ,1,0)
data_small["strongly_wrong"] =np.where(data_small['Sentiment']=='neutral' ,0,data_small["strongly_wrong"])
data_small["strongly_wrong"] =np.where(data_small['santiment_pred']=='neutral' ,0,data_small["strongly_wrong"])

print("right rate: ",data_small["right"].sum()/data_small["right"].count())
print("very wrong rate: ",data_small['strongly_wrong'].sum()/data_small["strongly_wrong"].count())

Some layers from the model checkpoint at nlptown/bert-base-multilingual-uncased-sentiment were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at nlptown/bert-base-multilingual-uncased-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


right rate:  0.6066666666666667
very wrong rate:  0.013333333333333334


In [ ]:
https://github.com/pysentimiento/pysentimiento

# play ground

In [11]:

model = TFAutoModelForSequenceClassification.from_pretrained("KernAI/stock-news-distilbert",from_pt=True)
tokenizer = AutoTokenizer.from_pretrained("KernAI/stock-news-distilbert")



All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [76]:
from transformers import pipeline

classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)
result = classifier(["The stock fell","The stock go up","apple lost case","terror group","win case","director die"])
print(result)


[{'label': 'negative', 'score': 0.9981594681739807}, {'label': 'positive', 'score': 0.46841657161712646}, {'label': 'negative', 'score': 0.9947216510772705}, {'label': 'neutral', 'score': 0.9445279836654663}, {'label': 'positive', 'score': 0.9798475503921509}, {'label': 'neutral', 'score': 0.9804892539978027}]


In [28]:
data = pd.read_csv("data_for_training_santiments.csv")

In [165]:
data_small = data.sample(500)

In [166]:
data_small["predict"] =data_small["Sentence"].apply(lambda sent: list(classifier(sent)[0].values()))

In [167]:
data_small["power"] = data_small['predict'].apply(lambda a: a[1])
data_small['predict'] = data_small['predict'].apply(lambda a: a[0])

In [136]:
data_small

,Sentence,Sentiment,predict,power
3485,Lassila & Tikanoja 's operating profit excludi...,negative,neutral,0.856572
4076,"In Finland , metal sector company Componenta i...",negative,negative,0.976236
1717,Cash flow from operations in January-December ...,negative,negative,0.998374
2054,$ROST Nov 90 calls 2.08 .. Go Long until exp i...,positive,neutral,0.997239
3769,Finnish meat company Atria can no longer promi...,negative,neutral,0.891702
5107,LONDON MarketWatch -- Share prices ended lower...,neutral,negative,0.997556
3161,"In addition , the contract includes modificati...",neutral,neutral,0.990817
4688,Originally posted to the PCMag.com security bl...,neutral,neutral,0.997328
1383,"Baird likes $TSLA - upgrades to outperform, li...",positive,positive,0.998509
3395,The long-standing partnership and commitment e...,positive,positive,0.987911


In [37]:
#data_small["predict"] = data_small["Sentence"].apply(classifier)

In [168]:
data_small["right"] = np.where(data_small['Sentiment']==data_small['predict'],1,0)
data_small["wrong"] = np.where(data_small['Sentiment']==data_small['predict'],0,1)

data_small["strongly_wrong"] =np.where(data_small['Sentiment']!=data_small['predict'] ,1,0)
data_small["strongly_wrong"] =np.where(data_small['Sentiment']=='neutral' ,0,data_small["strongly_wrong"])
data_small["strongly_wrong"] =np.where(data_small['predict']=='neutral' ,0,data_small["strongly_wrong"])


In [169]:
data_small["right"].sum()/data_small["right"].count()

0.62

In [170]:
print("right rate: ",data_small["right"].sum()/data_small["right"].count())
print("very wrong rate; ",data_small['strongly_wrong'].sum()/data_small["strongly_wrong"].count())

right rate:  0.62
very wrong rate;  0.024


In [171]:
strong = data_small[data_small['power']>data_small['power'].mean()+(1-data_small['power'].mean())*0.1]

In [172]:
print("right rate: ",strong["right"].sum()/strong["right"].count())
print("very wrong rate; ",strong['strongly_wrong'].sum()/strong["strongly_wrong"].count())

right rate:  0.6683804627249358
very wrong rate;  0.017994858611825194


In [178]:
data_small[data_small["right"]==0].groupby(['predict','Sentiment'])['Sentence'].count()

predict   Sentiment
negative  neutral      42
          positive      4
neutral   negative     11
          positive     26
positive  negative      8
          neutral      99
Name: Sentence, dtype: int64

In [179]:
classifier2 = pipeline("text-classification")
result = classifier2(["The stock fell","The stock go up","apple lost case","terror group","win case","director die"])
print(result)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


[{'label': 'NEGATIVE', 'score': 0.9996916055679321}, {'label': 'POSITIVE', 'score': 0.9924317598342896}, {'label': 'NEGATIVE', 'score': 0.999537467956543}, {'label': 'NEGATIVE', 'score': 0.9886809587478638}, {'label': 'POSITIVE', 'score': 0.999614953994751}, {'label': 'NEGATIVE', 'score': 0.9993342757225037}]


In [183]:
list(classifier2("The stock fell")[0].values())[0].lower()

'negative'

In [184]:
data_small["predict2"] =data_small["Sentence"].apply(lambda sent: list(classifier2(sent)[0].values()))

In [185]:
data_small["power2"] = data_small['predict2'].apply(lambda a: a[1])
data_small['predict2'] = data_small['predict2'].apply(lambda a: a[0].lower())

In [186]:
data_small

,Sentence,Sentiment,predict,power,right,wrong,strongly_wrong,predict2,power2
4068,Finnish security and privacy software solution...,positive,positive,0.997442,1,0,0,negative,0.934136
5412,"Isolationism? Another Sell Rating, Sell Rating...",negative,neutral,0.961804,0,1,0,negative,0.997624
2518,"Steve Jackson , eBusiness Analyst at Satama an...",neutral,neutral,0.606442,1,0,0,negative,0.812509
2318,Finnish automation solutions developer Cencorp...,neutral,negative,0.987838,0,1,0,negative,0.998935
5171,Finnish KCI Konecranes has raised its net sale...,positive,positive,0.997972,1,0,0,positive,0.993687
...,...,...,...,...,...,...,...,...,...
4891,HELSINKI ( AFX ) - Metso said it will upgrade ...,positive,positive,0.997210,1,0,0,positive,0.956265
1606,"The copying , republication or redistribution ...",neutral,negative,0.995548,0,1,0,negative,0.997213
2752,"Glaston , headquartered in Tampere , Finland ,...",neutral,positive,0.996659,0,1,0,positive,0.999345
4739,GlaxoSmithKline share price slips as FDA okays...,negative,negative,0.997758,1,0,0,negative,0.998802


In [37]:
#data_small["predict"] = data_small["Sentence"].apply(classifier)

In [188]:
data_small["right2"] = np.where(data_small['Sentiment']==data_small['predict2'],1,0)
data_small["wrong2"] = np.where(data_small['Sentiment']==data_small['predict2'],0,1)

data_small["strongly_wrong2"] =np.where(data_small['Sentiment']!=data_small['predict2'] ,1,0)
data_small["strongly_wrong2"] =np.where(data_small['Sentiment']=='neutral' ,0,data_small["strongly_wrong2"])
data_small["strongly_wrong2"] =np.where(data_small['predict2']=='neutral' ,0,data_small["strongly_wrong2"])


In [189]:
data_small["right2"].sum()/data_small["right2"].count()

0.308

In [190]:
print("right rate: ",data_small["right2"].sum()/data_small["right2"].count())
print("very wrong rate; ",data_small['strongly_wrong2'].sum()/data_small["strongly_wrong2"].count())

right rate:  0.308
very wrong rate;  0.15


In [191]:
strong = data_small[data_small['power2']>data_small['power2'].mean()+(1-data_small['power2'].mean())*0.1]

In [192]:
print("right rate: ",strong["right2"].sum()/strong["right2"].count())
print("very wrong rate; ",strong['strongly_wrong2'].sum()/strong["strongly_wrong2"].count())

right rate:  0.3582887700534759
very wrong rate;  0.12834224598930483


In [195]:
data_small.groupby(['predict2','Sentiment'])['Sentence'].count()

predict2  Sentiment
negative  negative      79
          neutral      165
          positive      73
positive  negative       2
          neutral      106
          positive      75
Name: Sentence, dtype: int64